TODO: add explanation on whats going on

In [1]:
import pandas as pd
import csv
df_dav = pd.read_csv('/content/df_davs.csv', sep=' ')
df_dav = df_dav.drop(["hate_speech", "count", "offensive_language", "neither"], axis = 1)
df_dav=df_dav.reindex(columns=['tweet', 'class'])
df_dav

,tweet,class
0,As a woman you shouldnt complain about cleani...,2
1,boy dats coldtyga dwn bad for cuffin dat hoe ...,1
2,Dawg RT You ever fuck a bitch and she start t...,1
3,she look like a tranny,1
4,The shit you hear about me might be true or i...,1
...,...,...
24778,yous a muthafin lie right His TL is trash Now ...,1
24779,youve gone and broke the wrong heart baby and ...,2
24780,young buck wanna eat dat nigguh like I aint fu...,1
24781,youu got wild bitches tellin you lies,1


In [2]:
df_was = pd.read_csv('/content/df_was.csv', sep=' ')
df_was["class_label"] = df_was['class_label'].fillna("none")
df_was['class_label'].loc[(df_was['class_label'] == "racism")] = 0
df_was['class_label'].loc[(df_was['class_label'] == "sexism")] = 1
df_was['class_label'].loc[(df_was['class_label'] == "none")] = 2
df_was['text'] = df_was['text'].astype(str)
df_was

,text,class_label
0,Drasko they didnt cook half a bird you idiot mkr,0
1,Hopefully someone cooks Drasko in the next ep ...,0
2,of course you were born in serbiayoure as fuck...,0
3,These girls are the equivalent of the irritati...,0
4,At least youre only a tiny bit racist Im not ...,0
...,...,...
2738,is above the national average wage PLUS its t...,1
2739,its a great website use it omg you are right ...,1
2740,this person thinks there is no wage gap smh N...,1
2741,Patriarchy will expel me if I divulge that I ...,1


In [3]:
import sklearn
from sklearn.model_selection import train_test_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [4]:
import torch
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
X_train, X_valid, Y_train, Y_valid= train_test_split(df_dav['tweet'].tolist(),
                                                      df_dav['class'].tolist(),
                                                      test_size=0.3)
train_data =list(zip(Y_train,X_train))
valid_data =list(zip(Y_valid,X_valid))

In [6]:
tzr = get_tokenizer('basic_english')
def torch_tokenizer(iterator):
  for _, text in iterator:
        yield tzr(text)

In [7]:
dataset_copy = train_data
transformed = build_vocab_from_iterator(torch_tokenizer(dataset_copy), specials=["<unk>"])
transformed.set_default_index(transformed["<unk>"])

In [8]:
text_to_index = lambda x: transformed(tzr(x))
label_trans = lambda x: int(x) 

In [9]:
text_to_index('hello world')

[1899, 329]

In [10]:
from torch import nn

class CNNModel1(nn.Module):

    def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel1, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.lin1 = nn.Linear(dim_in, 64)
        self.lin2 = nn.Linear(64,16)
        self.lin3 = nn.Linear(16, dim_out)
        self.relu = torch.nn.ReLU()
        self.init_weights()

    def init_weights(self):
        r = 0.5
        self.embedding.weight.data.uniform_(r, r)
        nn.init.xavier_uniform_(self.lin1.weight)
        nn.init.zeros_(self.lin1.bias)
        nn.init.kaiming_normal_(self.lin2.weight)
        nn.init.zeros_(self.lin2.bias)
        self.lin3.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.lin3.bias)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.relu(self.lin1(embedded))
        x = self.relu(self.lin2(x))
        x = self.lin3(x)
        return x

In [11]:
class CNNModel2(nn.Module):
      def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel2, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.relu = torch.nn.ReLU()
        self.linear_1 = torch.nn.Linear(dim_in, 100)
        self.linear_2 = torch.nn.Linear(100, 64)
        self.linear_3 = torch.nn.Linear(64, 16)
        self.linear_4 = torch.nn.Linear(16, dim_out)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.init_weights()

      def init_weights(self):
          r = 0.5
          self.embedding.weight.data.uniform_(r, r)
          nn.init.xavier_uniform_(self.linear_1.weight)
          nn.init.zeros_(self.linear_1.bias)
          nn.init.kaiming_normal_(self.linear_2.weight)
          nn.init.zeros_(self.linear_2.bias)
          self.linear_3.weight.data.uniform_(-r, r)
          nn.init.zeros_(self.linear_3.bias)
          self.linear_4.weight.data.uniform_(-r, r)
          nn.init.zeros_(self.linear_4.bias)

      def forward(self, text, offsets):
          embedded = self.embedding(text, offsets)
          x = self.relu(self.linear_1(embedded))
          x = self.relu(self.linear_2(x))
          x = self.dropout(self.linear_3(x))
          x = self.linear_4(x)
          return x

In [12]:
class CNNModel3(torch.nn.Module):
    def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel3, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.max_pool2d = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        self.batch_norm = torch.nn.BatchNorm1d(num_features=64)
        self.linear_1 = torch.nn.Linear(dim_in, 128)
        self.linear_2 = torch.nn.Linear(64, dim_out)
        self.relu = torch.nn.ReLU()
        self.init_weights()

    def init_weights(self):
        r = 0.5
        self.embedding.weight.data.uniform_(r, r)
        nn.init.kaiming_normal_(self.linear_2.weight)
        nn.init.zeros_(self.linear_2.bias)
        self.linear_1.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_1.bias)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.relu(self.linear_1(embedded))
        x = self.max_pool2d(x)
        x = self.batch_norm(x)
        x = x.reshape(x.size(0), -1) 
        x = self.linear_2(x)
        return x

In [13]:
class CNNModel4(torch.nn.Module):
    def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel4, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.conv_1 = torch.nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.max_pool2d = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        self.linear_1 = torch.nn.Linear(dim_in, 128)
        self.linear_2 = torch.nn.Linear(2048, 16)
        self.linear_3 = torch.nn.Linear(16, dim_out)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.relu = torch.nn.ReLU()
        self.init_weights()

    def init_weights(self):
        r = 0.5
        self.embedding.weight.data.uniform_(r, r)
        nn.init.xavier_uniform_(self.conv_1.weight)
        nn.init.zeros_(self.conv_1.bias)
        nn.init.kaiming_normal_(self.linear_2.weight)
        nn.init.zeros_(self.linear_2.bias)
        self.linear_1.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_1.bias)
        self.linear_3.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_3.bias)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        embedded = embedded.unsqueeze(1)
        x = self.relu(self.linear_1(embedded))
        x = self.relu(self.conv_1(x))
        x = self.max_pool2d(x)
        x = x.reshape(x.size(0), -1)
        x = self.relu(self.linear_2(x))
        x = self.dropout(x)
        x = self.linear_3(x)
        return x

In [14]:
class CNNModel5(torch.nn.Module):
    def __init__(self, transformer_size, dim_in, dim_out):
        super(CNNModel5, self).__init__()
        self.embedding = nn.EmbeddingBag(transformer_size, dim_in, sparse=True)
        self.conv_1 = torch.nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.max_pool2d = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        self.linear_1 = torch.nn.Linear(dim_in, 128)
        self.linear_2 = torch.nn.Linear(2048, 1024)
        self.linear_3 = torch.nn.Linear(1024, 512)
        self.linear_4 = torch.nn.Linear(512, 128)
        self.linear_5 = torch.nn.Linear(128, dim_out)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.relu = torch.nn.ReLU()
        self.init_weights()

    def init_weights(self):
        r = 0.5
        self.embedding.weight.data.uniform_(r, r)
        nn.init.xavier_uniform_(self.conv_1.weight)
        nn.init.zeros_(self.conv_1.bias)
        nn.init.kaiming_normal_(self.linear_2.weight)
        nn.init.zeros_(self.linear_2.bias)
        self.linear_1.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_1.bias)
        self.linear_3.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_3.bias)
        self.linear_4.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_4.bias)
        self.linear_5.weight.data.uniform_(-r, r)
        nn.init.zeros_(self.linear_5.bias)

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        embedded = embedded.unsqueeze(1)
        x = self.relu(self.linear_1(embedded))
        x = self.relu(self.conv_1(x))
        x = self.max_pool2d(x)
        x = x.reshape(x.size(0), -1)
        x = self.relu(self.linear_2(x))
        x = self.dropout(x)
        x = self.linear_3(x)
        x = self.relu(self.linear_4(x))
        x = self.linear_5(x)
        return x

In [15]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_trans(_label))
         processed_text = torch.tensor(text_to_index(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [16]:
import torch.optim as optim
def train_and_get_accuracy(train_d, valid_d, model, epochs = 10, lr = 0.001, BATCH_SIZE = 128):
  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)
  total_accu = None

  train_dataloader = DataLoader(train_d, batch_size=BATCH_SIZE, collate_fn=collate_batch)
  valid_dataloader = DataLoader(valid_d, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

  for epoch in range(epochs):
    # training step
    model.train()
    total_acc, total_count = 0, 0

    for idx, (label, text, offsets) in enumerate(train_dataloader):
      optimizer.zero_grad()
      predited_label = model(text, offsets)
      loss = criterion(predited_label, label)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
      optimizer.step()
      total_acc += (predited_label.argmax(1) == label).sum().item()
      total_count += label.size(0)

    # evaluation step
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(valid_dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

    accu_val = total_acc/total_count
    
  print('Final accuracy: {:f} '.format(accu_val))
  return accu_val

In [17]:
model = CNNModel1(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn1_accu_dav = train_and_get_accuracy(train_data, valid_data, model)
cnn1_accu_dav

Final accuracy: 0.773638 


0.7736381977135172

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = CNNModel2(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn2_accu_dav = train_and_get_accuracy(train_data, valid_data, model2, epochs = 100)
cnn2_accu_dav

Final accuracy: 0.773638 


0.7736381977135172

In [19]:
model = CNNModel3(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn3_accu_dav = train_and_get_accuracy(train_data, valid_data, model)
cnn3_accu_dav

Final accuracy: 0.783053 


0.7830531271015467

In [20]:
model = CNNModel4(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn4_accu_dav = train_and_get_accuracy(train_data, valid_data, model)
cnn4_accu_dav

Final accuracy: 0.773638 


0.7736381977135172

In [21]:
model = CNNModel5(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn5_accu_dav = train_and_get_accuracy(train_data, valid_data, model)
cnn5_accu_dav

Final accuracy: 0.773638 


0.7736381977135172

In [22]:
X_train, X_valid, Y_train, Y_valid= train_test_split(df_was['text'].tolist(),
                                                      df_was['class_label'].tolist(),
                                                      test_size=0.3)
train_data =list(zip(Y_train,X_train))
valid_data =list(zip(Y_valid,X_valid))

In [23]:
dataset_copy = train_data
transformed = build_vocab_from_iterator(torch_tokenizer(dataset_copy), specials=["<unk>"])
transformed.set_default_index(transformed["<unk>"])
text_to_index = lambda x: transformed(tzr(x))
label_trans = lambda x: int(x) 

In [24]:
model = CNNModel1(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn1_accu_was = train_and_get_accuracy(train_data, valid_data, model)
cnn1_accu_was

Final accuracy: 0.692588 


0.692588092345079

In [25]:
model = CNNModel2(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn1_accu_was = train_and_get_accuracy(train_data, valid_data, model)
cnn1_accu_was

Final accuracy: 0.003645 


0.0036452004860267314

In [26]:
model = CNNModel3(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn3_accu_was = train_and_get_accuracy(train_data, valid_data, model)
cnn3_accu_was

Final accuracy: 0.184690 


0.18469015795868773

In [27]:
model = CNNModel4(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn4_accu_was = train_and_get_accuracy(train_data, valid_data, model)
cnn4_accu_was

Final accuracy: 0.303767 


0.3037667071688943

In [28]:
model = CNNModel5(len(transformed), 1000, len(set([label for (label, text) in dataset_copy]))).to(device)
cnn5_accu_was = train_and_get_accuracy(train_data, valid_data, model)
cnn5_accu_was

Final accuracy: 0.003645 


0.0036452004860267314

Models on Waseem dataset perform poorly because there is only 2k instances. 

In [ ]:
# TODO: what if we shuffled the data a bit